# phitigra: a simple graph editor

[SageMath](https://www.sagemath.org/) has a large set of functions for [graph theory](https://doc.sagemath.org/html/en/reference/graphs/index.html). Defining graphs by hand can however be complicated as vertices and edges are added with the command line.
This package is an editor that allows to define or change graphs using the mouse. It has the form of a [Jupyter](https://jupyter.org/) widget.

## Getting started


In [ ]:
from src.phitigra import GraphEditor

The editor widget is a `GraphEditor` object. By default the canvas is empty; you can add vertices and edges by clicking on *add vertex or edge* and clicking on the canvas.

In [ ]:
editor = GraphEditor()
editor.show()

It is also possible to plot (and later edit) an already existing graph. Note that the two instances of the editor are completely independent.

In [ ]:
G = graphs.PetersenGraph()
editor2 = GraphEditor(G)
editor2.show()

Now you can move vertices, change their color, etc. The graph drawn can be accessed with `.graph`. It is the same object as the graph given when creating the widget.

In [ ]:
editor2.graph

In [ ]:
editor2.graph is G

A copy of the drawn graph can be obtained as follows:

In [ ]:
H = editor2.get_graph()
H == G and not H is G

### Application 1: testing a conjecture

In [ ]:
def conjecture(G):
    return not G.is_vertex_transitive() or G.is_hamiltonian()

Let us [conjecture](https://en.wikipedia.org/wiki/Lov%C3%A1sz_conjecture#Hamiltonian_cycle) that every vertex transitive graph is hamiltonian. Then `conjecture(G)` should return `True` for every graph `G`. We can to test in on various small graphs drawn in the widget. If the graph in the above widget is still the Petersen graph, the following should return `False`, disproving the conjecture.

In [ ]:
conjecture(editor2.get_graph())

### Application 2: producing pictures for your papers

The drawing of the graph in the editor can be exported to a latex (tikz) picture to be included in a paper. The latex code can be obtained as follows: 

In [ ]:
latex(editor2.graph)

Note that only the positions of the vertices will be kept. See [this page](https://doc.sagemath.org/html/en/tutorial/latex.html#an-example-combinatorial-graphs-with-tkz-graph) for more details about exporting graphs to latex . The resulting pdf image can be seen as follows.

In [ ]:
view(editor2.graph)

The above requires ``pdflatex``. It  will fail if you run this demo on binder.

## Widget settings

Several parameters of the widget can be changed:
  * the width and height of the drawing canvas;
  * the default radius and color for vertices;
  * the default color for edges;
  * whether or not the display vertex and edge labels.

In [ ]:
editor3 = GraphEditor(graphs.PetersenGraph(), width=300, height=300, default_radius=12, default_vertex_color='orange', default_edge_color='#666', show_vertex_labels=False)
editor3.show()

## Changing the drawing

Changes to the drawing can be done with the mouse of course, but also by calling appropriate functions.

### Automatically setting positions

In [ ]:
K = graphs.RandomBipartite(5,5,0.75)
editor4 = GraphEditor(K, width = 500, height = 500)
editor4.show()

The graph drawn above is bipartite and the partition numbe of a vertex is the first coordinate of its label. The code below sorts and colors vertices according to their partition number and changes their size according to the second coordinate of their label.

In [ ]:
for v in editor4.graph:
    p, i = v
    
    editor4.set_vertex_radius(v, 3 * i + 25)
    if p:
        editor4.set_vertex_pos(v, 100, 50 +  100 * i)
        # editor4.set_vertex_color(v, 'red')
    else:
        editor4.set_vertex_pos(v, 400, 50 + 100 * i)
        #editor4.set_vertex_color(v, 'lightblue')
    
editor4.refresh()                    # needed to update the canvas

The names of the vertices are by default the vertex labels, but can be changed by redefining the `get_vertex_label` function.

In [ ]:
def label(v):
    p, i = v
    return ('left ' if p else 'right ') + str(i)

editor4.get_vertex_label = label
editor4.refresh()

### Automatically setting colors

The colors of the vertices and edges can also be defined by a function.

In [ ]:
# Below we define a grid and delete many edges in it
n = 10
g = graphs.GridGraph([n,n])
for _ in range(5*n):
    e = g.random_edge()
    g.delete_edge(e)
    if not g.is_connected():
        g.add_edge(e)
    
editor5 = GraphEditor(g, default_radius=10, default_vertex_color='white', show_vertex_labels=False)
editor5.show()

The code below recolors the vertices depending on their distance to vertex `(3,1)`.

In [ ]:
def col(i, n):
    # Return a color depending on i
    rgbv = int(i * 255 / n)
    return '#%02x%02x%02x' % (100, rgbv , 255 - rgbv)

source = (3, 1)
distances = {v:g.distance(source, v) for v in g}
# maximum distance to (x,y) in the graph
max_dist = max(distances.values())

for v in g:
    d = distances[v]
    editor5.set_vertex_color(v, col(d, max_dist))
editor5.refresh()

## Running an algorithm step by step

Below we define a function that, when called on a `Graph Editor` widget and a source vertex, returns a generator. Each time an object is extracted from the generator (these objects are `None`), one step of Dijkstra's algorithm is executed on the graph of the widget and the colors and labels of the vertices are updated accordingly. We then define a button to trigger the runs of these steps. 

In [ ]:
def step_by_step_dijkstra(w, source):
    # Dijkstra's algorithm + some 'yield' statements to pause the algorithm at interesting times
    # and changes of colors of the graph edges and vertices.
    # Adapted from the pseudocode at https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm#Pseudocode

    G = w.graph
    Q = G.vertices()
    prev = {v: None for v in G}
    # We store distances in the widget, so that they can be accessed by the function
    # that returns vertex labels
    w.dist = {v: '?' for v in G}
    w.dist[source] = 0
    w.maxdist = 0
    
    while Q:
        u = Q[0]
        
        for v in Q:
            if w.dist[v] == '?':
                continue
            if w.dist[u] == '?' or w.dist[v] < w.dist[u]:
                u = v
        Q.remove(u)
        w._select_vertex(u, redraw=True)
        yield
        
        for v in G.neighbor_iterator(u):
            if v not in Q:
                continue
            alt = w.dist[u] + G.edge_label(u,v)
            if w.dist[v] == '?' or alt < w.dist[v]:
                # update
                if prev[v] is not None:
                    w.set_edge_color((v, prev[v]), 'cyan')
                
                w.dist[v] = alt
                w.maxdist = max(w.maxdist, alt)
                prev[v] = u
                w.set_vertex_color(v, 'green')
                w.set_edge_color((u,v), 'orange')
            else:
                w.set_edge_color((u,v), 'lightgray')
            w.refresh()
            yield
        w._select_vertex(u, redraw=True) # unselect
        w.done[u] = True
        w.refresh()

In [ ]:
g = graphs.GridGraph([5,5])
# Give random distances to edges
for u,v in g.edge_iterator(labels=False):
    g.set_edge_label(u,v, randint(0,50))

editor6 = GraphEditor(g, default_radius=20, default_vertex_color='white')
editor6.dist = {v: '?' for v in editor6.graph}
editor6.done = {v : False  for v in editor6.graph}

editor7.get_vertex_label = lambda v: str(editor7.dist[v])

button = Button(
    description='Next step',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='next step',
    icon='forward' # (FontAwesome names without the `fa-` prefix)
)

def color_label(w, v):
    if w.get_vertex_label(v) == '?':
        return 'lightpink'
    d = w.dist[v]
    maxd = w.maxdist
    if maxd:
        rgbv = int(d * 150/ maxd)
        if w.done[v]:
            return '#%02x%02x%02x' % (0, 255 - rgbv , 0)
        else:
            return '#%02x%02x%02x' % (255 - rgbv, 255 - rgbv , 255 - rgbv)
    else:
        return 'white'

editor6.get_vertex_color = lambda v: color_label(editor6, v)

gen = step_by_step_dijkstra(editor6, editor6.graph.random_vertex())

def button_clbk(b):
    try:
        next(gen)
    except StopIteration:
        b.disabled = True

# tie the button to button_clbk
button.on_click(button_clbk)

In [ ]:
editor6.show()

In [ ]:
button

## Animations

To make an animated canvas, one can simply ask the algorithm to _wait_ between the important steps. As an example, below is an animation for BFS.

In [ ]:
def widget_BFS(w, source):
    
    G = w.graph
    queue = [source]
    prev = {v: None for v in G}
    prev[source] = source
    
    while queue:
        
        # Take a new vertex in the queue
        v = queue.pop(0)
        w.set_vertex_color(v, 'red')
        w.refresh()
        yield
        
        # Add all its neighbors to the queue if they have not already been considered
        for u in w.graph.neighbor_iterator(v):
            if prev[u] is not None: # u has already been seen
                if prev[v] != u and not w.get_edge_color((u,v)) == 'lightgray':  
                    w.set_edge_color((u,v), 'lightgray')
                    w.refresh()
                    yield
            else:
                queue.append(u)
                prev[u] = v
                w.set_vertex_color(u, 'green')
                w.set_edge_color((u,v), 'orange')
                w.refresh()
                yield

        if v is source:
            w.set_vertex_color(v, 'purple')
        else:
            w.set_vertex_color(v, 'orange')
    w.set_vertex_color(v, 'orange')
    w.refresh()

In [ ]:
editor7 = GraphEditor(graphs.GridGraph([4,4]), default_radius=20, default_vertex_color='white', show_vertex_labels=False)
editor7.show()

In [ ]:
from time import sleep

def wait():
    sleep(float(0.5))
    
for _ in widget_BFS(editor7, source=editor7.graph.random_vertex()):
    wait()

This can even be used to produce animated images:

In [ ]:
editor8 = GraphEditor(graphs.GridGraph([5,5]), default_radius=20, default_vertex_color='white', show_vertex_labels=False)
editor8.show()

In [ ]:
# Adapted from https://www.geeksforgeeks.org/create-and-save-animated-gif-with-python-pillow

from PIL import Image

images = []
editor8._multi_canvas.sync_image_data = True

for _ in widget_BFS(editor8, source=editor8.graph.random_vertex()):
    wait()
    image_data = editor8._multi_canvas.get_image_data()
    image = Image.fromarray(image_data)
    images.append(image)


images[0].save('bfs.gif', save_all = True, append_images = images[1:], optimize = False, duration = 10)

